In [43]:
import glfw
import numpy as np
import math

from OpenGL.GL import *
from objs import *

# Configurações Iniciais 

---

In [44]:
# Inicialização da janela
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

# Configurações da janela
window = glfw.create_window(900, 900, "Cubo", None, None)
glfw.make_context_current(window)

# Interação com o Vertex Shaders do Pipeline Gráfico
vertex_code = """
            attribute vec3 position;
            uniform mat4 model;
            void main(){
                gl_Position = model * vec4(position,1.0);
            }
        """
        
# Interação com o Fragment Shaders do Pipeline Gráfico
fragment_code = """
            uniform vec4 color;
            void main(){
                gl_FragColor = color;
            }
        """

# Requisitando slots para a GPU para os programas acima
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Associação do código fonte aos slots solicitados
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compila o Vertex Shader
glCompileShader(vertex)

# Em caso de erros encerra o programa
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

# Compila o Fragment Shader
glCompileShader(fragment)

# Em caso de erros encerra o programa
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Associação dos programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Link dos programas
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Definição de nosso programa como padrão
glUseProgram(program)

# Funções úteis

---

In [45]:
def get_eye_matrix():
    matrix = np.array([
        1.0, 0.0, 0.0, 0.0,
        0.0, 1.0, 0.0, 0.0,
        0.0, 0.0, 1.0, 0.0,
        0.0, 0.0, 0.0, 1.0
    ], np.float64)

    return matrix

def get_rotate_matrix_x(angle):
    cos = math.cos(angle)
    sin = math.sin(angle)
    
    matrix = np.array([
        1.0, 0.0,  0.0, 0.0,
        0.0, cos, -sin, 0.0,
        0.0, sin,  cos, 0.0,
        0.0, 0.0,  0.0, 1.0
    ], np.float64)
    
    return matrix

def get_rotate_matrix_y(angle):
    cos = math.cos(angle)
    sin = math.sin(angle)
    
    matrix = np.array([
         cos, 0.0, sin, 0.0,
         0.0, 1.0, 0.0, 0.0,
        -sin, 0.0, cos, 0.0,
         0.0, 0.0, 0.0, 1.0
    ], np.float64)
    
    return matrix

def get_rotate_matrix_z(angle):
    cos = math.cos(angle)
    sin = math.sin(angle)
    
    matrix = np.array([
        cos, -sin, 0.0, 0.0,
        sin,  cos, 0.0, 0.0,
        0.0,  0.0, 1.0, 0.0,
        0.0,  0.0, 0.0, 1.0
    ], np.float64)

    
    return matrix

def get_translade_matrix(tx, ty, tz):
    matrix = np.array([
        1.0, 0.0, 0.0,  tx,
        0.0, 1.0, 0.0,  ty,
        0.0, 0.0, 1.0,  tz,
        0.0, 0.0, 0.0, 1.0
    ], np.float64)
    
    return matrix

def get_scale_matrix(sx, sy, sz):
    matrix = np.array([
         sx, 0.0, 0.0, 0.0,
        0.0,  sy, 0.0, 0.0,
        0.0, 0.0, sz, 0.0,
        0.0, 0.0, 0.0, 1.0
    ], np.float64)

    return matrix

In [46]:
def mul_matrix(matrix_a, matrix_b):
    matrix_a = matrix_a.reshape(4, 4)
    matrix_b = matrix_b.reshape(4, 4)
    matrix_dot = np.dot(matrix_a, matrix_b)
    
    return matrix_dot.flatten()

In [47]:
def model(tx, ty, tz, rx, ry, rz, sx, sy, sz):
    
    matrix_transform = get_eye_matrix()
    
    matrix_translade = get_translade_matrix(tx, ty, tz)
    matrix_transform = mul_matrix(matrix_transform, matrix_translade)
    

    matrix_rotate_x = get_rotate_matrix_x(math.radians(rx))
    matrix_transform = mul_matrix(matrix_transform, matrix_rotate_x)


    matrix_rotate_y = get_rotate_matrix_y(math.radians(ry))
    matrix_transform = mul_matrix(matrix_transform, matrix_rotate_y)
    

    matrix_rotate_z = get_rotate_matrix_z(math.radians(rz))
    matrix_transform = mul_matrix(matrix_transform, matrix_rotate_z)
        
    matrix_scale = get_scale_matrix(sx, sy, sz)
    matrix_transform = mul_matrix(matrix_transform, matrix_scale)
        
    return matrix_transform

In [48]:
class Object3D:
    def __init__(self):
        self.tx, self.ty, self.tz = 0.0, 0.0, 0.0  
        self.rx, self.ry, self.rz = 0.0, 0.0, 0.0  
        self.sx, self.sy, self.sz = 1.0, 1.0, 1.0  
        self.rotate = True
        self.select = False
        self.angle = 0.05

    def set_coordinates(self, tx, ty, tz, rx, ry, rz, sx, sy, sz):
        self.tx, self.ty, self.tz = tx, ty, tz
        self.rx, self.ry, self.rz = rx, ry, rz
        self.sx, self.sy, self.sz = sx, sy, sz

    def update_translate(self, tx, ty, tz):
        self.tx += tx
        self.ty += ty
        self.tz += tz

    def update_rotate(self, rx, ry, rz):
        if self.rotate:
            self.rx += rx
            self.ry += ry
            self.rz += rz

    def update_scale(self, sx, sy, sz):
        self.sx += sx
        self.sy += sy
        self.sz += sz
    
    def set_select(self, select):
        self.select = select

    def reset(self):
        self.__init__()


In [49]:
class Hearth(Object3D):
    def __init__(self):
        super().__init__() 
        self.tx, self.ty, self.tz = 0.4, 0.3, 0.0
        self.sx, self.sy, self.sz = 0.18, 0.18, 0.18

        self.colors = [
            [(1.0, 0.0, 0.0, 1.0), (1.0, 0.5, 0.31, 1.0)],
            [(1.0, 0.84, 0.0, 1.0), (1.0, 1.0, 0.0, 1.0)]
        ]

    def draw_object(self):
        mat_model = model(self.tx, self.ty, self.tz, 
                          self.rx, self.ry, self.rz, 
                          self.sx, self.sy, self.sz)
        
        loc_model = glGetUniformLocation(program, "model")
        loc_color = glGetUniformLocation(program, "color")
        
        glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
        
        glUniform4f(loc_color, 
                    self.colors[self.select][0][0], 
                    self.colors[self.select][0][1], 
                    self.colors[self.select][0][2], 
                    self.colors[self.select][0][3])
        for i in range(0, 30, 5):  
            glDrawArrays(GL_TRIANGLE_STRIP, SET_POSITION_HEARTHS + i, 5)


        glUniform4f(loc_color, 
                    self.colors[self.select][1][0], 
                    self.colors[self.select][1][1], 
                    self.colors[self.select][1][2], 
                    self.colors[self.select][1][3])
        for i in range(32, 56, 3):  
            glDrawArrays(GL_TRIANGLE_STRIP, SET_POSITION_HEARTHS + i, 3)

In [50]:
class Spades(Object3D):
    def __init__(self):
        super().__init__() 
        self.tx, self.ty, self.tz = 0.40, -0.50, 0.00
        self.sx, self.sy, self.sz = 0.15,  0.15, 0.15
    
        self.colors = [
            [(0.0, 0.0, 0.0, 1.0), (0.1, 0.1, 0.1, 1.0)],
            [(1.0, 0.84, 0.0, 1.0), (1.0, 1.0, 0.0, 1.0)]
        ]
    
    def draw_object(self):
        mat_model = model(self.tx, self.ty, self.tz, 
                          self.rx, self.ry, self.rz, 
                          self.sx, self.sy, self.sz)
        
        loc_model = glGetUniformLocation(program, "model")
        loc_color = glGetUniformLocation(program, "color")
        
        glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
        
        glUniform4f(loc_color, 
                    self.colors[self.select][0][0], 
                    self.colors[self.select][0][1], 
                    self.colors[self.select][0][2], 
                    self.colors[self.select][0][3])
        for i in range(0, 12, 3):
            glDrawArrays(GL_TRIANGLE_STRIP, SET_POSITION_SPADES + i, 3)

        glUniform4f(loc_color, 
                    self.colors[self.select][1][0], 
                    self.colors[self.select][1][1], 
                    self.colors[self.select][1][2], 
                    self.colors[self.select][1][3])
        for i in range(12, 36, 4):
            glDrawArrays(GL_TRIANGLE_STRIP, SET_POSITION_SPADES + i, 4)

In [51]:
class Diamonds(Object3D):
    def __init__(self):
        super().__init__() 
        self.tx, self.ty, self.tz = -0.40, -0.20, 0.00
        self.sx, self.sy, self.sz =  0.20,  0.30, 0.20
        
        self.colors = [
                [(1.0, 0.0, 0.0, 1.0), (1.0, 0.5, 0.31, 1.0)],
                [(1.0, 0.84, 0.0, 1.0), (1.0, 1.0, 0.0, 1.0)]
            ]
    
    def draw_object(self):
        mat_model = model(self.tx, self.ty, self.tz, 
                          self.rx, self.ry, self.rz, 
                          self.sx, self.sy, self.sz)
        
        loc_model = glGetUniformLocation(program, "model")
        loc_color = glGetUniformLocation(program, "color")
        
        glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
        
        glUniform4f(loc_color, 
                    self.colors[self.select][0][0], 
                    self.colors[self.select][0][1], 
                    self.colors[self.select][0][2], 
                    self.colors[self.select][0][3])
        for i in range(0, 12, 3):
            glDrawArrays(GL_TRIANGLE_STRIP, SET_POSITION_DIAMONDS + i, 3)

        glUniform4f(loc_color, 
                    self.colors[self.select][1][0], 
                    self.colors[self.select][1][1], 
                    self.colors[self.select][1][2], 
                    self.colors[self.select][1][3])
        for i in range(12, 24, 3):
            glDrawArrays(GL_TRIANGLE_STRIP, SET_POSITION_DIAMONDS + i, 3)

# Dados que serão utilizados

---

In [52]:
# Envia os dados para a GPU
buffer = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Envia os todos os dados de vértices
glBufferData(GL_ARRAY_BUFFER, VERTEXES.nbytes, VERTEXES, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Associa as variáveis do programa GLSL com os dados
stride = VERTEXES.strides[0]
offset = ctypes.c_void_p(0)

# Localiza todos os vértices de position
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

# Indicados a GPU onde está o conteúdo
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

# Janela

---

In [53]:
hearths = Hearth()
spades = Spades()
diamonds = Diamonds()

tx, ty = 0.0, 0.0
rx, ry = 0.0, 0.0
sx, sy = 0.0, 0.0

wireframe_mode = False

select_obj = None

In [54]:
def key_event(window,key,scancode,action,mods):
    global select_obj, wireframe_mode
    global tx, ty, rx, ry, sx, sy
    
    if action == glfw.PRESS:
        if key == glfw.KEY_1:
            select_obj = hearths
        elif key == glfw.KEY_2:
            select_obj = spades
        elif key == glfw.KEY_3:
            select_obj = diamonds
                            
        if select_obj != None:
            select_obj.set_select(1)
    
            for obj in [hearths, spades, diamonds]:
                if obj != select_obj:
                    obj.set_select(0)
                                
        if key == glfw.KEY_R:
            if select_obj != None:
                select_obj.rotate = not select_obj.rotate
                
        if key == glfw.KEY_P:
            if wireframe_mode:
                glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)  
            else:
                glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)
                
            wireframe_mode = not wireframe_mode
            
    tx, ty = 0.0, 0.0
    rx, ry = 0.0, 0.0
    sx, sy = 0.0, 0.0
    
    if action == glfw.PRESS or action == glfw.REPEAT:
        
        if key == glfw.KEY_W:  
            ty = +0.001
        elif key == glfw.KEY_S:
            ty = -0.001
        elif key == glfw.KEY_A:
            tx = -0.001
        elif key == glfw.KEY_D:  
            tx = +0.001
        
        if key == glfw.KEY_UP:  
            sy = +0.001
        elif key == glfw.KEY_DOWN:
            sy = -0.001
        elif key == glfw.KEY_LEFT:
            sx = -0.001
        elif key == glfw.KEY_RIGHT:  
            sx = +0.001
            
        if key == glfw.KEY_PAGE_UP:  
            ry = +0.05
        elif key == glfw.KEY_PAGE_DOWN:
            ry = -0.05
            
        if select_obj != None:
            select_obj.angle += ry
            

glfw.set_key_callback(window,key_event)

In [55]:
glfw.show_window(window)

In [56]:
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):
    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    
    glClearColor(1.0, 1.0, 1.0, 1.0)

    if select_obj != None:  
        select_obj.update_translate(tx=tx, ty=ty, tz=0.0)
        select_obj.update_scale(sx=sx, sy=sy, sz=0.0)
        select_obj.draw_object()
        
    
    for obj in [hearths, spades, diamonds]:
        obj.update_rotate(rx=0.0, ry=obj.angle, rz=0.0)
        obj.draw_object()
    
    glfw.swap_buffers(window)


glfw.terminate()